## EXTRACCIÓN DE NOTICIAS

In [12]:
import urllib
from bs4 import BeautifulSoup
import requests
import numpy
import os
import json

In [13]:
url_general = 'https://maldita.es/malditobulo/1'
html_general = requests.get(url_general).text
soup_general = BeautifulSoup(html_general, 'html.parser')

In [14]:
clase_nav_bar = 'bg-white text-gray-800 px-3 py-2 text-lg border-gray-400 font-bold'

pags_link = soup_general.find('span', attrs={'class' : clase_nav_bar})('a')[0].get('href', None)
num_pags = int(pags_link.split('/')[-1])
print(num_pags)

560


In [15]:
bulos = {}

In [16]:
clase_new = 'section-card-headline'
cont = 0
for i in range(1, num_pags+1):
    url = 'https://maldita.es/malditobulo/'+str(i)
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')
    for div in soup.find_all('div',attrs={'class' : clase_new}):
        a = div('a')[0]
        title = a.text
        link = a.get('href', None)
        bulos[cont] = {"titulo" : title, "link" : link}
        cont += 1

In [46]:
bulos

{0: {'titulo': 'No, no están regalando llaveros con localizador para luego robarte o secuestrarte',
  'link': 'https://maldita.es/malditobulo/20211130/regalando-llaveros-localizador-robarte-secuestrarte/'},
 1: {'titulo': 1, 'link': 1},
 2: {'titulo': 'No, no es cierto que "todo lo que has publicado" en Facebook "se convierte en público desde hoy, incluso los mensajes y las fotos que han sido borrados", a fecha de 29 de noviembre de 2021 *',
  'link': 'https://maldita.es/malditobulo/20211129/facebook-publico-mensajes-fotos-eliminados/'},
 3: {'titulo': 'La imagen que mezcla una declaración de Pérez-Reverte con un texto titulado "un país bananero" del que no hay pruebas de que el escritor sea el autor',
  'link': 'https://maldita.es/malditobulo/20211129/texto-pais-bananero-arturo-perez-reverte/'},
 4: {'titulo': "Cuidado con este supuesto sorteo de Lidl de una aspiradora Dyson: es 'phishing'",
  'link': 'https://maldita.es/malditobulo/20211129/sorteo-aspiradora-dyson-phishing-suscripcio

In [18]:
len(bulos)

5597

## ALMACENANDO NOTICIAS

In [48]:
file = open("fakenewsMaldita.json", "w")
json.dump(bulos, file)
file.close()